# <span style="color:darkblue"> Lecture 16 - Practicing Chaining </span>

<font size = "5">

In this lecture you will get a chance to practice <br>
the main dataset operations

- There will be a quiz on this lecture

# <span style="color:darkblue"> I. Import Libraries and Data </span>


In [2]:
import numpy as np
import pandas as pd

In [3]:
results  = pd.read_csv("data_raw/results.csv")
races    = pd.read_csv("data_raw/races.csv")
results["points col"] = results["points"]

# <span style="color:darkblue"> II. Review Dataset Operations </span>

<font size = "5">

See attached file for a refresher on syntax

```[] ``` $\qquad \qquad \qquad \quad$: Extracting columns <br>
```.query() ``` $\qquad \qquad $: Subsetting rows <br>
```.recode() ``` $ \qquad \quad \ \ $: Replacing values <br>
```.groupby().agg() ```: Aggregate statistics by subgroup <br>
```.rename() ``` $\qquad \quad \ \ $: Change name of columns

Full list:

<font size = "4">

https://www.w3schools.com/python/pandas/pandas_ref_dataframe.asp

# <span style="color:darkblue"> III. Examples of Chaining </span>

<font size = "5">

The operations with "." are read left to right

- Combine any of the above operations
- Great way to make code efficient
- The sky's the limit!


Subsetting **before** extracting columns

In [4]:
# Get data for drivers that scored more than 20 points on individual races
# Then extract the columns "driverId" and "points"
results.query('points >= 20')[["driverId","points"]]

,driverId,points
20320,4,25.0
20344,18,25.0
20368,20,25.0
20392,18,25.0
20416,17,25.0
...,...,...
25740,830,25.0
25760,830,25.0
25780,830,25.0
25800,847,26.0


<font size = "5">

Subsetting **before** aggregating

In [5]:
# This obtains a subset of drivers who competed in races 500 onwards
# then computes the average by team ("constructorId")

(results.query('raceId >= 500')
        .groupby("constructorId")
        .agg(mean_points = ("points","mean")))


,mean_points
constructorId,
1,3.148148
3,1.904924
4,1.903226
5,1.203911
6,4.910966
...,...
209,0.012821
210,0.809028
211,3.723684


<font size = "5">

Subsetting **after** aggregating

In [6]:
# This obtains the average points by team ("constructorId"), then 
# produces a subset of team whose average is higher than 10

(results.groupby("constructorId")
        .agg(mean_points = ("points","mean"))
        .query('mean_points >= 10'))

,mean_points
constructorId,
131,12.363643


<font size = "5">

Chaining inside queries + NaNs

In [ ]:
# "is.na()" produces a True/False vector, checking for missing values
# "is.notna()" produces a True/False vector, checking for non-missing values
# .str.isnumeric() is used for checking whether individual rows of a
# string column are numeric.
results["points"].isna()
results["points"].notna()

subset_nas    = results.query('points.isna()')
subset_nonnas = results.query('points.notna()')


## <span style="color:darkblue"> III. Quiz Structure </span>

<font size = "5">

The day of the quiz I will ...
- Provide a dataset with information
- Give more specific instructions.
- Below, you will see the type of questions that will be asked.
- The idea is for you to apply known concepts to new data
- You have 50 minutes to complete the assignment

Questions

(exact wording may change in quiz, but exercise will be very similar)



<font size = "5">

(a) Replace the values of a column

- Obtain unique string values of a column
- Use the ".replace()" command

Hint: See Lecture 13

In [ ]:
# "list_old" encodes values we want to change
# "list_new" encodes the values that will "replace" the old
list_old = ['\\N','-7']
list_new = [np.nan,-7]

# This command replaces the values of the "alt" column
circuits["alt"] = circuits["alt"].replace(list_old, list_new)

# Note: The option "inplace = True" permanently modifies the column
# circuits["alt"].replace(list_old, list_new, inplace = True)

circuits["country"] = circuits["country"].replace("UK","United Kingdom")
circuits["country"] = circuits["country"].replace(["UK"],["United Kingdom"])

<font size = "5">

(b) Recode a numeric column

- Use the "pd.cut()" command to create <br>
a new column based on an interval.
- See Lecture 14 for more details

In [ ]:
bins_x = [0,2500, 5000]
labels_x = ["Between 0 and 2500",
            "Between 2500 and 5000"]

circuits["bins_alt"] = pd.cut(circuits["alt_numeric"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

# Note: if we set bins_x = [float("-inf"),2500, float("inf")]
#       then intervals are "Less than or equal to 2500" and "Above 2500"
#       float("inf") and float("-inf") represent infinity and negative infinity
#       The "right" command indicates that the right interval is
#       "less than or equal to" or just "less than"


bins_hemisphere = [-90,0,90]
labels_hemisphere = ["Southern hemisphere","Northern Hemisphere"]
circuits["hemisphere"] = pd.cut(circuits["lat"],
                                bins = bins_hemisphere,
                                right = True,
                                labels = labels_hemisphere)

<font size = "5">

(c) Aggregate and query

- Use a combniation of the following commands <br>
to produce a new dataset <br>
``` .query() ``` <br>
``` .groupby().agg() ``` <br>

In [ ]:
# The following gets a subset of the data using .query()
# In this case we subset the data before computing aggregate statistics
# Note: "filtering" is often the word used to obtain a subset

teamrace_agg = (results.query("raceId >= 500")
                       .groupby(["raceId","constructorId"])
                        .agg(mean_points = ('points','mean'),
                             sd_points =   ('points','std'),
                             min_points =  ('points','min'),
                             max_points =  ('points','max'),
                             count_obs   = ('points',len)))


# Write your own code

pointslaps_agg = (results.groupby("raceId")
                         .agg(mean_points = ("points","mean"),
                              mean_laps = ("laps","mean")))

len(pointslaps_agg)

<font size = "5">

(d) Aggregate and sort

- Use a combniation of the following commands <br>
to produce a new dataset <br>
``` .groupby().agg() ``` <br>
``` .sort_values() ```

In [ ]:
constructor_agg = (results.groupby("constructorId")
                          .agg(mean_points = ("points","mean"))
                          .sort_values("mean_points",ascending = False))


constructor_agg

<font size = "5">

(e) Rename column

- Create a dictionary
- Rename one or more columns in a dataset <br>
using the dictionary

In [ ]:
# This is an example of a pandas data frame created from a dictionary
# This example illustrates the basic syntax of a dictionary

car_dictionary = {"car_model": ["Ferrari","Tesla","BMW"],
                  "year":      ["2018","2023","2022"] }

pd.DataFrame(car_dictionary)

# We first define the dictionary
# Change the pipe ".rename(...)" to rename the columns
# Dictionaries can flexibly accommodate single values or list after ":"

dict_rename_circuits = {"name": "circuit_name"}
circuits = circuits_raw.rename(columns = dict_rename_circuits)

print("Old List:")
print(circuits_raw.columns.values)
print("")
print("New List:")
print(circuits.columns.values)

dict_rename_races = { "name": "race_name" }
races = races_raw.rename(columns = dict_rename_races)

<font size = "5">

(f) Merge dataset

- Use "pd.merge" to combine two datasets: <br>
a primary and secondary
- Only merge a subset of the columns of the <br>
secondary dataset
- Use "display" to show a the merged dataset,  <br>
extracting a subset of the columns

In [ ]:
# The "pd.merge()" command combines the information from both datasets
# The first argument is the "primary" datasets
# The second argument is the "secondary" dataset (much include the "on" column)
# The "on" is the common variable that is used for merging
# how = "left" tells Python that the left dataset is the primary one

races_merge = pd.merge(races_raw,
                       circuits[["circuitId","circuit_name"]],
                       on = "circuitId",
                       how = "left")

races_merge[["raceId","circuitId","circuit_name"]].sort_values(by = "circuit_name")

results_merge = pd.merge(results_raw,
                         races_raw[["raceId","date"]],
                         on = "raceId",
                         how = "left")

